Transfer learning using cryptobert and roberta

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig, TextClassificationPipeline
from transformers import pipeline
import numpy as np
import pandas as pd
from scipy.special import softmax
import datetime

/Users/endy/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

## Step 1 - Importing Bitcoin Tweet Data to begin analysing the model

In [4]:
# Kaggle - 2021Q1
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q1_reduced.csv')

# Kaggle - 2021Q2
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q3
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

# Kaggle - 2021Q4
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_2021Q2_reduced.csv')

In [196]:
# Kaggle - 202101, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202101_reduced.csv')
# Kaggle - 202102, issue
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202102_reduced.csv')
# Kaggle - 202103, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202103_reduced.csv')
# Kaggle - 202104, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202104_reduced.csv')
# Kaggle - 202105, issue
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202105_reduced.csv')
# Kaggle - 202106, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202106_reduced.csv')
# Kaggle - 202107, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202107_reduced.csv')
# Kaggle - 202108, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202108_reduced.csv')
# Kaggle - 202109, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202109_reduced.csv')
# Kaggle - 202110, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202110_reduced.csv')
# Kaggle - 202111, done
# data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202111_reduced.csv')
# Kaggle - 202112
data = pd.read_csv('~/Code/giadapi/crypto/data/raw/tweets_202112_reduced.csv')

In [197]:
data

,Unnamed: 0,datetime,username,text,has_#bitcoin,date
0,187925,2021-12-01 22:09:59+00:00,cryptowizar_d,@IndraDEX_ @PurposeInvest I would like to get ...,True,2021-12-01
1,187926,2021-12-01 22:09:58+00:00,Spcowins,@spectatorindex #Bitcoin solves this.,True,2021-12-01
2,187927,2021-12-01 22:09:40+00:00,onlykingjunior,❇️ @MinersDefi second Buyback and and Burn eve...,True,2021-12-01
3,187928,2021-12-01 22:09:38+00:00,NakamotoIs,@CoinDesk @saylor inevitable #Bitcoin,True,2021-12-01
4,187929,2021-12-01 22:09:35+00:00,BlockchainedX,2 Hottest Solana And Ethereum Projects (Ready ...,True,2021-12-01
...,...,...,...,...,...,...
14892,202817,2021-12-31 06:00:00+00:00,tulipan81,Bitcoin blocks size distribution per pool in l...,True,2021-12-31
14893,202818,2021-12-31 06:00:00+00:00,tulipan81,Bitcoin blocks delay in last day https://t.co/...,True,2021-12-31
14894,202819,2021-12-31 06:00:00+00:00,tulipan81,Bitcoin mined blocks distribution per pool in ...,True,2021-12-31
14895,202820,2021-12-31 06:00:00+00:00,tulipan81,Bitcoin mined blocks distribution per pool in ...,True,2021-12-31


In [198]:
data = data[['text', 'date']]

In [199]:
data

,text,date
0,@IndraDEX_ @PurposeInvest I would like to get ...,2021-12-01
1,@spectatorindex #Bitcoin solves this.,2021-12-01
2,❇️ @MinersDefi second Buyback and and Burn eve...,2021-12-01
3,@CoinDesk @saylor inevitable #Bitcoin,2021-12-01
4,2 Hottest Solana And Ethereum Projects (Ready ...,2021-12-01
...,...,...
14892,Bitcoin blocks size distribution per pool in l...,2021-12-31
14893,Bitcoin blocks delay in last day https://t.co/...,2021-12-31
14894,Bitcoin mined blocks distribution per pool in ...,2021-12-31
14895,Bitcoin mined blocks distribution per pool in ...,2021-12-31


## Step 2 - Cleaning the data

In [111]:
# I have changed this to remove more information

def preprocess(text):
    new_text = []
    text = str(text)
    text = text.replace("\n", " ")
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [200]:
#Create a dummy data
data['process_text'] = data.text
data['negative_bert'] = data.text
data['neutral_bert'] = data.text
data['positive_bert'] = data.text

#use the preprocess to clean the data
data['process_text'] = data['text'].apply(preprocess)

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_1960/1973951134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_1960/1973951134.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['negative_bert'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_1960/1973951134.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [201]:
data

,text,date,process_text,negative_bert,neutral_bert,positive_bert
0,@IndraDEX_ @PurposeInvest I would like to get ...,2021-12-01,I would like to get some of this #bitcoin ETF!,@IndraDEX_ @PurposeInvest I would like to get ...,@IndraDEX_ @PurposeInvest I would like to get ...,@IndraDEX_ @PurposeInvest I would like to get ...
1,@spectatorindex #Bitcoin solves this.,2021-12-01,#Bitcoin solves this.,@spectatorindex #Bitcoin solves this.,@spectatorindex #Bitcoin solves this.,@spectatorindex #Bitcoin solves this.
2,❇️ @MinersDefi second Buyback and and Burn eve...,2021-12-01,❇️ second Buyback and and Burn event has happ...,❇️ @MinersDefi second Buyback and and Burn eve...,❇️ @MinersDefi second Buyback and and Burn eve...,❇️ @MinersDefi second Buyback and and Burn eve...
3,@CoinDesk @saylor inevitable #Bitcoin,2021-12-01,inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin
4,2 Hottest Solana And Ethereum Projects (Ready ...,2021-12-01,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...
...,...,...,...,...,...,...
14892,Bitcoin blocks size distribution per pool in l...,2021-12-31,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...
14893,Bitcoin blocks delay in last day https://t.co/...,2021-12-31,Bitcoin blocks delay in last day #BTC #bitcoi...,Bitcoin blocks delay in last day https://t.co/...,Bitcoin blocks delay in last day https://t.co/...,Bitcoin blocks delay in last day https://t.co/...
14894,Bitcoin mined blocks distribution per pool in ...,2021-12-31,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...
14895,Bitcoin mined blocks distribution per pool in ...,2021-12-31,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...


## Step 3 - analyse the language and sentiments by pretrained model

In [151]:
!pyenv local crypto

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
#Model 2b - Use the bert model, with full scores
MODEL_bert = f"ElKulako/cryptobert"
tokenizer_bert = AutoTokenizer.from_pretrained(MODEL_bert)
tokenizer_bert.model_max_length = 512 #solve the error: RuntimeError: The expanded size of the tensor (562) must match the existing size (514) at non-singleton dimension
config_bert = AutoConfig.from_pretrained(MODEL_bert)


# PT
model_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_bert)
model_bert.config.max_position_embeddings = 512

In [66]:
def scores_bert(sample_text):
    encoded_input_bert = tokenizer_bert(sample_text, return_tensors='pt')
    output_bert = model_bert(**encoded_input_bert)
    scores_bert = output_bert[0][0].detach().numpy()
    scores_bert = softmax(scores_bert) #1st score is negative, 2nd score is netural, 3rd score is positive
    return scores_bert

In [67]:
# starting_num = 0
# ending_num = 100

# for i in range(2):
#     try:
#         csv1 = data['process_text'][starting_num : ending_num]
#         result_csv = csv1.concat(pd.DataFrame(csv1.apply(scores_bert)),axis=1)
#         print(f'{starting_num} to {ending_num} worked!')
#         file_name = f"{starting_num}_{ending_num}_twitter_comments.csv"
#         result_csv.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")
#         print(f'{starting_num} to {ending_num} saved!')
#     except:
#         print(f'{starting_num} to {ending_num} has not processed correctly')
#     starting_num += 100
#     ending_num += 100

In [202]:
data['text'] = data['process_text'].apply(scores_bert)

In [203]:
data

,text,date,process_text,negative_bert,neutral_bert,positive_bert
0,"[0.00041044387, 0.54253644, 0.45705312]",2021-12-01,I would like to get some of this #bitcoin ETF!,@IndraDEX_ @PurposeInvest I would like to get ...,@IndraDEX_ @PurposeInvest I would like to get ...,@IndraDEX_ @PurposeInvest I would like to get ...
1,"[0.007571444, 0.532546, 0.4598826]",2021-12-01,#Bitcoin solves this.,@spectatorindex #Bitcoin solves this.,@spectatorindex #Bitcoin solves this.,@spectatorindex #Bitcoin solves this.
2,"[9.773075e-05, 0.6933666, 0.3065357]",2021-12-01,❇️ second Buyback and and Burn event has happ...,❇️ @MinersDefi second Buyback and and Burn eve...,❇️ @MinersDefi second Buyback and and Burn eve...,❇️ @MinersDefi second Buyback and and Burn eve...
3,"[0.0044133514, 0.14657061, 0.8490161]",2021-12-01,inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin,@CoinDesk @saylor inevitable #Bitcoin
4,"[0.00022758059, 0.6344284, 0.36534405]",2021-12-01,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...,2 Hottest Solana And Ethereum Projects (Ready ...
...,...,...,...,...,...,...
14892,"[0.004841147, 0.66908675, 0.32607213]",2021-12-31,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...,Bitcoin blocks size distribution per pool in l...
14893,"[0.87886554, 0.058289684, 0.06284478]",2021-12-31,Bitcoin blocks delay in last day #BTC #bitcoi...,Bitcoin blocks delay in last day https://t.co/...,Bitcoin blocks delay in last day https://t.co/...,Bitcoin blocks delay in last day https://t.co/...
14894,"[0.0036264313, 0.771306, 0.22506759]",2021-12-31,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...
14895,"[0.0090717515, 0.7450273, 0.24590094]",2021-12-31,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...,Bitcoin mined blocks distribution per pool in ...


In [204]:
for i in range(len(data)):    
    data['negative_bert'][i] = data['text'][i][0]
    data['neutral_bert'][i] = data['text'][i][1]
    data['positive_bert'][i] = data['text'][i][2]

In [205]:
data = data[['date', 'process_text', 'negative_bert', 'neutral_bert','positive_bert']]

In [206]:
data

,date,process_text,negative_bert,neutral_bert,positive_bert
0,2021-12-01,I would like to get some of this #bitcoin ETF!,0.00041,0.542536,0.457053
1,2021-12-01,#Bitcoin solves this.,0.007571,0.532546,0.459883
2,2021-12-01,❇️ second Buyback and and Burn event has happ...,0.000098,0.693367,0.306536
3,2021-12-01,inevitable #Bitcoin,0.004413,0.146571,0.849016
4,2021-12-01,2 Hottest Solana And Ethereum Projects (Ready ...,0.000228,0.634428,0.365344
...,...,...,...,...,...
14892,2021-12-31,Bitcoin blocks size distribution per pool in l...,0.004841,0.669087,0.326072
14893,2021-12-31,Bitcoin blocks delay in last day #BTC #bitcoi...,0.878866,0.05829,0.062845
14894,2021-12-31,Bitcoin mined blocks distribution per pool in ...,0.003626,0.771306,0.225068
14895,2021-12-31,Bitcoin mined blocks distribution per pool in ...,0.009072,0.745027,0.245901


## Step 4: Count the total number of labels/scores (positive, negative vs neutral) by date

In [207]:
grouped_data = pd.DataFrame(data.groupby(['date'])[['negative_bert', 'neutral_bert', 'positive_bert']].sum().fillna(0).reset_index())
grouped_data = grouped_data.rename_axis("", axis="columns")

In [208]:
grouped_data

,date,negative_bert,neutral_bert,positive_bert
0,2021-12-01,33.079273,334.769775,253.150803
1,2021-12-02,29.942642,304.941711,242.115707
2,2021-12-03,48.248924,314.411469,239.339539
3,2021-12-04,111.281281,466.295471,434.422882
4,2021-12-05,35.309681,278.630463,234.059875
5,2021-12-06,56.319256,336.050903,305.629791
6,2021-12-07,39.934368,238.508957,175.556717
7,2021-12-08,34.813095,242.072983,179.113892
8,2021-12-09,46.616699,245.123703,173.259750
9,2021-12-10,50.885303,257.110504,205.004272


## Step 5 - Download the data

In [209]:
start_date = grouped_data['date'][0]
# start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")
start_date

'2021-12-01'

In [210]:
end_date = grouped_data['date'][len(grouped_data)-1]
# end_date_str = datetime.datetime.strftime(end_date, "%Y-%m-%d")
end_date

'2021-12-31'

In [211]:
file_name = f"{start_date}_{end_date}_twitter_comments.csv"
data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")

In [212]:
file_name = f"{start_date}_{end_date}_twitter_transferlearning.csv"
grouped_data.to_csv(f"~/code/giadapi/crypto/data/processed/{file_name}")